In [108]:
full_info = {}

In [109]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [110]:
import docx

def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        txt = para.text.encode('ascii', 'ignore')
        fullText.append(txt)
    return '\n'.join(fullText)

In [111]:
document = getText('../docs/doc_2.docx')

In [112]:
import re
lines_ini = re.split('    |\n',document)

In [113]:
lines = [nltk.word_tokenize(el) for el in lines_ini]
lines = [nltk.pos_tag(el) for el in lines]
lines = [x for x in lines if x != []]

In [114]:
grammer = r'NAME: {^<NN.*><NN.*><NN.*>?$}'
chunkParser = nltk.RegexpParser(grammer)

In [115]:
nameHits = []
for tagged_tokens in lines:
    chunked_tokens = chunkParser.parse(tagged_tokens)
    for subtree in chunked_tokens.subtrees():
        if subtree.label() == 'NAME':
            nameHits.append(' '.join([i[0] for i in subtree.leaves()]))

In [116]:
full_info['Candidate Name'] = nameHits[0]
full_info['offerer Name'] = nameHits[-1]

In [117]:
import spacy
nlp = spacy.load('en')

def extract_join_dt_sal_org(lines):
    dates = []
    money = []
    org = []
    for line in lines:
        doc=nlp(unicode(line))
        for ent in doc.ents:
            if ent.label_ == 'ORG':
                    org.append(str(ent.text))
            if ent.label_ == 'MONEY':
                    money.append(str(ent.text))
            if ent.label_ == 'DATE':
                    match = re.search(r'(^\d{1,2}\D.*\d{4}$)',ent.text.strip())
                    if match != None:
                        dates.append(str(ent.text))
    
    if not dates:
        dates = ['not found']
    if not money:
        money = ['not found']
    if not org:
        org = ['not found']
    return dates[-1],money[0],org[0]

In [118]:
join_date,salary,org=extract_join_dt_sal_org(lines_ini)
full_info['Joining date'] = join_date
full_info['Salary'] = salary
full_info['Organisation Name'] = org

In [119]:
import find_job_titles
from find_job_titles import FinderAcora
b=FinderAcora()
gen=FinderAcora.find_raw(b,unicode(document))
designation=[]
for l in gen:
    designation.append(l)
    break

In [120]:
des= designation[0]
d = document[des.start-15:des.end+15]

In [121]:
d = nltk.word_tokenize(str(d))
d = nltk.pos_tag(d)
grammer = r'DESIGNATION: {<NNP>*}'
chunkParser = nltk.RegexpParser(grammer)
desHits = []
chunked_tokens = chunkParser.parse(d)
for subtree in chunked_tokens.subtrees():
    if subtree.label() == 'DESIGNATION':
        word=' '.join([i[0] for i in subtree.leaves()])
        if des.match in word:
            desHits.append(word)

In [122]:
full_info['Designation'] = desHits[0]

In [123]:
full_info

{'Candidate Name': 'Bob Jostle',
 'Designation': 'Assistant Cashier Manager',
 'Joining date': '11/2/2014',
 'Organisation Name': 'The Number Store',
 'Salary': '31,000',
 'offerer Name': 'Dave Thimble'}